In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.ensemble import *
from datetime import datetime
from tensorflow.keras.callbacks import *

C:\Users\Lenovo\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv("./soccer-spi/spi_matches.csv")

In [3]:
df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def dateconv (dates):
  date = datetime.strptime(dates,'%Y-%m-%d') 
  return date

df['date'] = df['date'].apply(lambda x : dateconv(x))

In [54]:
last_played = datetime(2020,9,10)
GW_end = datetime(2020,9,23)
pl = df[df['league']== "Barclays Premier League"]
old = pl[pl['date']<last_played]
new = pl[pl['date']>last_played]
new = new[new['date'] < GW_end]

In [55]:
teams = pd.get_dummies(new[['team1','team2']])
teamsold = pd.get_dummies(old[['team1','team2']])

In [56]:
new = pd.concat([new,teams], axis=1)
old = pd.concat([old,teamsold], axis=1)

In [57]:
new.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,team2_Liverpool,team2_Manchester City,team2_Manchester United,team2_Newcastle,team2_Sheffield United,team2_Southampton,team2_Tottenham Hotspur,team2_West Bromwich Albion,team2_West Ham United,team2_Wolverhampton
34052,2020,2020-09-11,2411,Barclays Premier League,Manchester City,Aston Villa,94.11,66.48,0.8888,0.0251,...,0,0,0,0,0,0,0,0,0,0
34053,2020,2020-09-11,2411,Barclays Premier League,Burnley,Manchester United,69.08,86.83,0.1919,0.5707,...,0,0,1,0,0,0,0,0,0,0
34058,2020,2020-09-12,2411,Barclays Premier League,Fulham,Arsenal,64.18,77.16,0.2817,0.4608,...,0,0,0,0,0,0,0,0,0,0
34104,2020,2020-09-12,2411,Barclays Premier League,Crystal Palace,Southampton,67.69,72.48,0.3658,0.3571,...,0,0,0,0,0,1,0,0,0,0
34114,2020,2020-09-12,2411,Barclays Premier League,Liverpool,Leeds United,91.07,64.09,0.8383,0.0427,...,0,0,0,0,0,0,0,0,0,0


In [58]:
old_teams = old.columns.tolist()
new_teams = new.columns.tolist()

In [59]:
dropped_columns_old = []
dropped_columns_new = []
for column in old_teams:
    if column not in new_teams:
        dropped_columns_old.append(column)
for column in new_teams:
    if column not in old_teams:
        dropped_columns_new.append(column)

In [61]:
dropped_columns_new

['team1_Leeds United', 'team2_Leeds United']

In [62]:
X_train = old.drop(['date','league_id','league','team1','team2','score1','score2','adj_score1','adj_score2','xg1','xg2','nsxg1','nsxg2','importance1','importance2','team1_AFC Bournemouth',
 'team1_Cardiff City',
 'team1_Huddersfield Town',
 'team1_Hull City',
 'team1_Middlesbrough',
 'team1_Norwich City',
 'team1_Stoke City',
 'team1_Sunderland',
 'team1_Swansea City',
 'team1_Watford',
 'team2_AFC Bournemouth',
 'team2_Cardiff City',
 'team2_Huddersfield Town',
 'team2_Hull City',
 'team2_Middlesbrough',
 'team2_Norwich City',
 'team2_Stoke City',
 'team2_Sunderland',
 'team2_Swansea City',
 'team2_Watford',
],axis=1)
X_test = new.drop(['date','league_id','league','team1','team2','score1','score2','adj_score1','adj_score2','xg1','xg2','nsxg1','nsxg2','importance1','importance2', 'team1_Leeds United',
 'team2_Leeds United'],axis=1)

In [63]:
lm = LinearRegression()
lm.fit(X_train,old[['score1','score2']])
predictions  = lm.predict(X_test)
preds = []
for num in predictions:
  score1 = np.round(num[0])
  score2 = np.round(num[1])
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsPLGW1GW2_LR.xlsx")

20
20


In [64]:
tX_train = X_train.values
tX_test =X_test.values
y_train = old[['score1','score2']].values

In [65]:
scaler = MinMaxScaler()
tX_train = scaler.fit_transform(tX_train)
tX_test = scaler.transform(tX_test)
tX_train.shape

(1520, 46)

In [66]:
model = Sequential()

In [67]:
model.add(Flatten())
model.add(Dense(64,activation=tf.nn.relu))
model.add(Dense(128,activation=tf.nn.relu))
model.add(Dense(256,activation=tf.nn.relu))
model.add(Dense(2))


In [68]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [69]:
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=25)

In [70]:
model.fit(tX_train,y_train,epochs=300,callbacks=[early_stop])

Train on 1520 samples
Epoch 1/300
1520/1520 [==============================] - 0s 256us/sample - loss: 1.7566 - accuracy: 0.7105
Epoch 2/300
1520/1520 [==============================] - 0s 67us/sample - loss: 1.3235 - accuracy: 0.7191
Epoch 3/300
1520/1520 [==============================] - 0s 78us/sample - loss: 1.2990 - accuracy: 0.7178
Epoch 4/300
1520/1520 [==============================] - 0s 72us/sample - loss: 1.2715 - accuracy: 0.7118
Epoch 5/300
1520/1520 [==============================] - 0s 78us/sample - loss: 1.2532 - accuracy: 0.7197
Epoch 6/300
1520/1520 [==============================] - 0s 79us/sample - loss: 1.2372 - accuracy: 0.7303
Epoch 7/300
1520/1520 [==============================] - 0s 88us/sample - loss: 1.2097 - accuracy: 0.7336
Epoch 8/300
1520/1520 [==============================] - 0s 72us/sample - loss: 1.1994 - accuracy: 0.7355
Epoch 9/300
1520/1520 [==============================] - 0s 79us/sample - loss: 1.1770 - accuracy: 0.7336
Epoch 10/300
1520/1520 

1520/1520 [==============================] - 0s 94us/sample - loss: 0.4785 - accuracy: 0.8395
Epoch 78/300
1520/1520 [==============================] - 0s 72us/sample - loss: 0.4585 - accuracy: 0.8395
Epoch 79/300
1520/1520 [==============================] - 0s 68us/sample - loss: 0.4705 - accuracy: 0.8375
Epoch 80/300
1520/1520 [==============================] - 0s 69us/sample - loss: 0.4627 - accuracy: 0.8447
Epoch 81/300
1520/1520 [==============================] - 0s 66us/sample - loss: 0.4509 - accuracy: 0.8375
Epoch 82/300
1520/1520 [==============================] - 0s 70us/sample - loss: 0.4439 - accuracy: 0.8454
Epoch 83/300
1520/1520 [==============================] - 0s 67us/sample - loss: 0.4482 - accuracy: 0.8395
Epoch 84/300
1520/1520 [==============================] - 0s 65us/sample - loss: 0.4277 - accuracy: 0.8539
Epoch 85/300
1520/1520 [==============================] - 0s 66us/sample - loss: 0.4384 - accuracy: 0.8467
Epoch 86/300
1520/1520 [==========================

1520/1520 [==============================] - 0s 97us/sample - loss: 0.2710 - accuracy: 0.8724
Epoch 154/300
1520/1520 [==============================] - 0s 91us/sample - loss: 0.2801 - accuracy: 0.8618
Epoch 155/300
1520/1520 [==============================] - 0s 86us/sample - loss: 0.2622 - accuracy: 0.8632
Epoch 156/300
1520/1520 [==============================] - 0s 91us/sample - loss: 0.2730 - accuracy: 0.8579s - loss: 0.2638 - accuracy: 0.
Epoch 157/300
1520/1520 [==============================] - 0s 96us/sample - loss: 0.2756 - accuracy: 0.8717
Epoch 158/300
1520/1520 [==============================] - 0s 90us/sample - loss: 0.2720 - accuracy: 0.8632
Epoch 159/300
1520/1520 [==============================] - 0s 121us/sample - loss: 0.2735 - accuracy: 0.8651
Epoch 160/300
1520/1520 [==============================] - 0s 141us/sample - loss: 0.2686 - accuracy: 0.8664
Epoch 161/300
1520/1520 [==============================] - 0s 266us/sample - loss: 0.2636 - accuracy: 0.8664
Epoch 16

1520/1520 [==============================] - 0s 97us/sample - loss: 0.1918 - accuracy: 0.8750
Epoch 229/300
1520/1520 [==============================] - 0s 92us/sample - loss: 0.1807 - accuracy: 0.8757
Epoch 230/300
1520/1520 [==============================] - 0s 129us/sample - loss: 0.1815 - accuracy: 0.8770
Epoch 231/300
1520/1520 [==============================] - 0s 96us/sample - loss: 0.1885 - accuracy: 0.8724
Epoch 232/300
1520/1520 [==============================] - 0s 163us/sample - loss: 0.1921 - accuracy: 0.8743
Epoch 233/300
1520/1520 [==============================] - 0s 140us/sample - loss: 0.1986 - accuracy: 0.8638
Epoch 234/300
1520/1520 [==============================] - 0s 191us/sample - loss: 0.1859 - accuracy: 0.8697
Epoch 235/300
1520/1520 [==============================] - 0s 150us/sample - loss: 0.1805 - accuracy: 0.8789
Epoch 236/300
1520/1520 [==============================] - 0s 157us/sample - loss: 0.1779 - accuracy: 0.8750
Epoch 237/300
1520/1520 [===========

In [72]:
predictions  = model.predict(tX_test)
preds = []
for num in predictions:
  score1 = np.round(num[0]) if num[0]>=0 else 0
  score2 = np.round(num[1]) if num[1]>=0 else 0
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsPLGW1GW2_TF.xlsx")

20
20
